# IBDB Webscraping
This file webscrapes publically available data from the Broadway League's IBDB (Internet Broadway Database) for the purposes of analyzing the importance of Tony Awards outcomes for Broadway Productions.

This file borrows code and takes general inspiration from the following [Colaboratory Jupyter Notebook](https://colab.research.google.com/drive/1IVwOhBMYay14NkO7kGkrPu0Ij9dSDdEP) by Yaakov Bressler

In [1]:
import io
import requests
from lxml import etree
from bs4 import BeautifulSoup
import urllib.request
import re
import string
#import time
import datetime
import pandas as pd
import numpy as np
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Create a function that grabs links from a page, using a tag to identify value of link

In [2]:
def getLinks_tagged_fast(url, tag):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    links = []
    # set the opening of each link to be...
    tag = tag
    for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
        links.append(link.get('href'))
    return links

https://www.broadwayworld.com/browseshows.cfm?showtype=BR

The above link is your starting point. It will allow us to get the name of every Broadway production that opened between 1979 and now.

In [3]:
def get_show_links_year(year_url):
    """
    This function...
    """
    url = year_url
    tag_year = 'browseshows.cfm?'
    #calling previous function to get the link I want
    years = getLinks_tagged_fast(url, tag_year)[1:]
    page_base = 'https://www.broadwayworld.com/'
    years_loop =[]
    for year in years:
        #focusing on 1979 or later
        if year[-4:].isdigit() and int(year[-4:]) >= 1979:
            years_loop.append(page_base+year)
    
    # Now you have all the years
    tag_show = 'https://www.broadwayworld.com/shows/backstage.php?'
    show_links_nested = []
    for year in years_loop:
        show_links_nested.append(getLinks_tagged_fast(year,tag_show))
    show_links = sum(show_links_nested, [])
    
    return show_links
    
#running function to get my list of links to the productions I want
year_url = 'https://www.broadwayworld.com/browseshows.cfm?showtype=BR'
show_links = get_show_links_year(year_url)

In [4]:
#don't need every show ever, just 1979 onwards
#from here, you will the scrape name of each show, so you can search for it on ibdb.com using selenium
#print(show_links)
#the number of productions between 1979 and now fits the estimate I made in another assignment (roughly 1800)
len(show_links)

1877

In [5]:
def get_show_name(url):
    """
    This function...
    """
    response = requests.get(url)
    #make sure url actually exists
    assert response.status_code == 200

    show_html = response.text
    htmlparser = etree.HTMLParser()
    tree = etree.parse(io.StringIO(show_html), parser=htmlparser)
    showroot = tree.getroot()
    #this xml path is the same for every url, it will give you the name of the show for that url
    show_name = showroot.xpath("//span[@itemprop = 'name']/text()")[1]
    
    return show_name

In [6]:
#testing the function on a random url (tried this with several urls)
get_show_name('https://www.broadwayworld.com/shows/backstage.php?showid=6366')

'Oliver!'

In [7]:
#using get_show_name function to make a list of every single show I want to find data on
#took about 10 minutes to run this chunk
show_names = []
for link in show_links:
    show_name = get_show_name(link)
    show_names.append(show_name)
    
len(show_names)

1877

In [14]:
#due to the presence of revivals in the list, some show names appear more than once
#therefore, to reduce redundancy when I search for these shows on the database, I am only keeping unique show names
unique_show_names = []
for show in show_names:
    if show not in unique_show_names:
        unique_show_names.append(show)

#print(unique_show_names)     
len(unique_show_names)

1590

In [12]:
#show_names

In [9]:
#testing functions from selenium package 
#will delete this chunk later
driver = webdriver.Chrome()
productions = ['The King and I', 'Dear Evan Hansen', 'Spring Awakening']
for prod in productions:
    driver.get('https://www.ibdb.com/shows/')     #website I am searching from (IBDB)
    e = driver.find_element('name','ShowProperName')    #locating the searchbar
    e.send_keys(prod)       #automating the searches (will store results in a list later)
    e.send_keys(Keys.ENTER)
    
#^MIGHT MAKE MORE SENSE TO SEARCH FOR PRODUCTIONS FROM ADVANCED SEARCH PAGE, RATHER THAN SEARCH FOR SHOWS

In [16]:
#testing out process for grabbing hrefs I need from search results
#next step is to limit it so it only grabs hrefs for productions post 1979
resp = requests.get('https://www.ibdb.com/broadway-show/the-king-and-i-5067') #don't need '#nyc-production' in url
#resp = requests.get('https://www.ibdb.com/broadway-show/sunday-in-the-park-with-george-8418')
test_html = resp.text
htmlparser = etree.HTMLParser()
tree = etree.parse(io.StringIO(test_html), parser=htmlparser)
testroot = tree.getroot()
links = testroot.xpath('//div[@id = "nyc-productions" and @data-id = "nyc-productions"]/div/div/div/div/a/@href')

links

['/broadway-production/the-king-and-i-1935',
 '/broadway-production/the-king-and-i-3999',
 '/broadway-production/the-king-and-i-4357',
 '/broadway-production/the-king-and-i-4673',
 '/broadway-production/the-king-and-i-497593']

In [77]:
#trying this with BS instead
soup = BeautifulSoup(prod_html, 'html.parser')
# this one works too
# script = soup.find('script', attrs={'type':'text/javascript'})
script = soup.find_all('script', type='text/javascript')
#js: str = script.text.replace('\n', '')
#raw_json = re.search('var spConfig=newApex.Config\(({.*})\);', js, flags=re.MULTILINE).group(1)
#data = json.loads(raw_json)
#labels = [opt['label'] for opt in data['attributes']['199']['options']]
#print(labels)
script[1]

<script type="text/javascript">
        $(document).ready(function () {
            Navigation.init(null);

            var gallery = new Gallery($('#gallery'));
            gallery.Init();

            var amazonLightModal = new LightModal('#amazon-links');
            amazonLightModal.Init();

            $('#amazon-popup').click(function () {
                amazonLightModal.Show();
            });

            var audioLightModal = new AudioLightModal('#audio-player','audio-iframe');
            audioLightModal.Init();

            $('#audio-popup').click(function () {
                audioLightModal.Show();
            });

            $('.owl-gallery').owlCarousel({
                items: 10,
                dots: false,
                nav: true,
                autoplayTimeout: 2500,
                loop: false,
                video: true,
                rewind: true,
                navText: ['<i class="material-icons">chevron_left</i>', '<i class="material-icons">chevron_ri

In [45]:
url = 'https://www.ibdb.com'
js_list = []
for frag in links:
    prod_link = url + frag
    resp = requests.get(prod_link)
    prod_html = resp.text
    prod_tree = etree.parse(io.StringIO(prod_html), parser=htmlparser)
    prod_root = tree.getroot()
    js_elt = prod_root.xpath('//script[@type = "text/javascript" and position() = 2]')[0]
    js_list.append(js_elt)

In [59]:
type(js_list[0])

lxml.etree._Element

In [10]:
#need to get "/broadway-production/<name-of-show-number>" for every production
#href is important
#URL = "https://www.ibdb.com/shows/" 